In [ ]:
import torch
import gc

gc.collect()
torch.cuda.empty_cache()

In [ ]:
#mount the drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Need to have higher version of pyarrow

In [ ]:
!pip install --upgrade pyarrow

     |████████████████████████████████| 17.7MB 198kB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


## Clone the hugging face transformers code

In [ ]:
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 48362 (delta 32), reused 42 (delta 21), pack-reused 48299
Receiving objects: 100% (48362/48362), 35.57 MiB | 13.28 MiB/s, done.
Resolving deltas: 100% (33706/33706), done.


In [ ]:
cd transformers

/content/transformers


In [ ]:
!ls


CODE_OF_CONDUCT.md  hubconf.py	 notebooks	 setup.py   valohai.yaml
CONTRIBUTING.md     LICENSE	 pyproject.toml  src
docker		    Makefile	 README.md	 templates
docs		    MANIFEST.in  scripts	 tests
examples	    model_cards  setup.cfg	 utils


###Install all the required libraries needed to run the transformers. After installation restart the kernel to enable the effect of newly installed or upgraded libraries

In [ ]:
!pip install .
!pip install -r ./examples/requirements.txt

Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.1MB 7.1MB/s 
     |████████████████████████████████| 2.9MB 23.5MB/s 
     |████████████████████████████████| 890kB 35.8MB/s 
  Created wheel for transformers: filename=transformers-3.4.0-cp36-none-any.whl size=1278393 sha256=a9856d9e8c814d1d22c6cb72f53694a8a7ac4803c1d3408e9e7059f479f40743
  Stored in directory: /tmp/pip-ephem-wheel-cache-dwghrecn/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=5f3cda93b62de1786b23b6ab6e35b732647d6faf58e04f4d4b5fb7cc22b56b03
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 51kB 3.2MB/s 

In [ ]:
!pwd

/content


In [ ]:
!dir /content/transformers/examples/

adversarial	     lightning_base.py	 seq2seq
benchmarking	     longform-qa	 test_examples.py
bert-loses-patience  lxmert		 test_xla_examples.py
bertology	     movement-pruning	 text-classification
conftest.py	     multiple-choice	 text-generation
contrib		     question-answering  token-classification
deebert		     rag		 xla_spawn.py
distillation	     README.md
language-modeling    requirements.txt


## Copy the 10K file <- to be used as input for fine tuning the Bert LM model

In [ ]:
cd "/content/drive/My Drive/praveen_iiit_hyd_ire_class/ten_k_project_work"

/content/drive/My Drive/praveen_iiit_hyd_ire_class/ten_k_project_work


In [ ]:
!mkdir "/input_data"
!mkdir "/outputs"

In [ ]:
!cp "/content/drive/My Drive/praveen_iiit_hyd_ire_class/ten_k_project_work/tenk_input_old.txt" /input_data
!du -s -h /input_data/*

4.3M	/input_data/tenk_input_old.txt


In [ ]:
!head -n 20 /input_data/tenk_input_old.txt

213898 GrossFileSize
NetFileSize 137834
HTML_Chars 836
hdr 19940829 sgml
CONFORMED 10-K SUBMISSION TYPE
DOCUMENT PUBLIC COUNT
CONFORMED PERIOD 19940531 REPORT
19940824 DATE FILED
NAME CONFORMED COMPANY AAR CORP
CENTRAL INDEX KEY 1750
STANDARD INDUSTRIAL CLASSIFICATION 5080
IRS 362334820 NUMBER
YEAR END FISCAL 531
TYPE FORM 10-K
Act 1934 ACT SEC
FILE NUMBER SEC 1-06263
FILM NUMBER 94545798
1111 NICHOLAS BLVD STREET
CITY VILLAGE ELK GROVE
7084393939 BUSINESS PHONE


In [ ]:
!ls /content/transformers/examples/language-modeling

chinese_ref.py	README.md  run_language_modeling.py


# Execute fine tuning

In [ ]:
import timeit

start = timeit.default_timer() 

!python /content/transformers/examples/language-modeling/run_language_modeling.py \
    --output_dir=/outputs \
    --model_type=bert \
    --model_name_or_path=bert-base-uncased \
    --do_train \
    --train_data_file=/input_data/tenk_input_old.txt \
    --do_eval \
    --eval_data_file=/input_data/tenk_input_old.txt \
    --num_train_epochs=1 \
    --save_steps=400 \
    --mlm

stop = timeit.default_timer()
print (stop - start)


2020-10-28 14:19:35.042983: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
10/28/2020 14:19:37 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
10/28/2020 14:19:37 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='/outputs', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Oct28_14-19-37_250bd44e4e29', logging_first_ste

In [ ]:
!du -s -h /outputs/*

4.0K	/outputs/config.json
4.0K	/outputs/eval_results_lm.txt
418M	/outputs/pytorch_model.bin
4.0K	/outputs/special_tokens_map.json
4.0K	/outputs/tokenizer_config.json
4.0K	/outputs/training_args.bin
228K	/outputs/vocab.txt


## Copy the fine tuned model to the folder <- this new model will be used in all the downstream intrinsic and extrinsic tasks

In [ ]:
!cp /outputs/* "/content/drive/My Drive/praveen_iiit_hyd_ire_class/ten_k_project_work/output/"

In [ ]:
import glob
import os
from transformers import WEIGHTS_NAME
print(WEIGHTS_NAME)
directory_cpt = list(os.path.dirname(c) for c in sorted(glob.glob("/outputs" + "/**/" + WEIGHTS_NAME, recursive=True)))
print(directory_cpt)

pytorch_model.bin
['/outputs']


In [ ]:
!head -n 10 /outputs/vocab.txt

[PAD]
[unused0]
[unused1]
[unused2]
[unused3]
[unused4]
[unused5]
[unused6]
[unused7]
[unused8]


In [ ]:
!head -n 10 /outputs/special_tokens_map.json

{"unk_token": "[UNK]", "sep_token": "[SEP]", "pad_token": "[PAD]", "cls_token": "[CLS]", "mask_token": "[MASK]"}

In [ ]:
!head -40 /outputs/config.json

{
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}


In [ ]:
# Load cached data and see
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("/outputs")
print(tokenizer.encode("assets component acquired Company liabilities aircraft assumed substantially 2000 support Hermetic certain company"))

[101, 7045, 6922, 3734, 2194, 22393, 14680, 2948, 5071, 12381, 2456, 2490, 2014, 11368, 2594, 3056, 2194, 102]


In [ ]:
from transformers import pipeline
help(pipeline)

Help on function pipeline in module transformers.pipelines:

pipeline(task:str, model:Optional=None, config:Union[str, transformers.configuration_utils.PretrainedConfig, NoneType]=None, tokenizer:Union[str, transformers.tokenization_utils.PreTrainedTokenizer, NoneType]=None, framework:Union[str, NoneType]=None, use_fast:bool=False, **kwargs) -> transformers.pipelines.Pipeline
    Utility factory method to build a :class:`~transformers.Pipeline`.
    
    Pipelines are made of:
    
        - A :doc:`tokenizer <tokenizer>` in charge of mapping raw textual input to token.
        - A :doc:`model <model>` to make predictions from the inputs.
        - Some (optional) post processing for enhancing model's output.
    
    Args:
        task (:obj:`str`):
            The task defining which pipeline will be returned. Currently accepted tasks are:
    
            - :obj:`"feature-extraction"`: will return a :class:`~transformers.FeatureExtractionPipeline`.
            - :obj:`"sentiment-ana

In [ ]:
fill_mask = pipeline(
    "fill-mask",
    model="/outputs",
    tokenizer="/outputs"
)

Some weights of BertModel were not initialized from the model checkpoint at /outputs and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Do a quick check of the new model on fill in the blank task

In [ ]:
result = fill_mask( "Company would materially occurred [MASK] results operations different 2001 beginning acquisition")
result

[{'score': 0.027501985430717468,
  'sequence': '[CLS] company would materially occurred financial results operations different 2001 beginning acquisition [SEP]',
  'token': 3361,
  'token_str': 'financial'},
 {'score': 0.022655580192804337,
  'sequence': '[CLS] company would materially occurred following results operations different 2001 beginning acquisition [SEP]',
  'token': 2206,
  'token_str': 'following'},
 {'score': 0.017097260802984238,
  'sequence': '[CLS] company would materially occurred company results operations different 2001 beginning acquisition [SEP]',
  'token': 2194,
  'token_str': 'company'},
 {'score': 0.016840597614645958,
  'sequence': '[CLS] company would materially occurred the results operations different 2001 beginning acquisition [SEP]',
  'token': 1996,
  'token_str': 'the'},
 {'score': 0.01670217514038086,
  'sequence': '[CLS] company would materially occurred without results operations different 2001 beginning acquisition [SEP]',
  'token': 2302,
  'token